In [1]:
BASE_DIR ='../'*3
UTIL_DIR = BASE_DIR + 'code/UTIL/math/mathutil.ipynb'
%run {UTIL_DIR}

Using device: cuda
NVIDIA GeForce RTX 3060 Laptop GPU
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB
Using PyTorch version: 1.10.1


In [18]:
from torch.utils.data import DataLoader, Dataset

In [19]:
class Dataset_torch(object):
    def __init__(self, name, mode):
        self.name = name
        self.mode = mode
#         self.time_serial = False
    
    def __str__(self):
        return '{}({}, {}+{}+{})'.format(self.name, self.mode, \
                   len(self.tr_xs), len(self.te_xs), len(self.va_xs))

    @property
    def train_count(self):
        return len(self.tr_xs)

In [ ]:
def dataset_get_train_data(self, batch_size, nth):
    from_idx = nth * batch_size
    to_idx = (nth + 1) * batch_size
    tr_X = self.tr_xs[self.indices[from_idx:to_idx]]
    tr_Y = self.tr_ys[self.indices[from_idx:to_idx]]
    
    return tr_X, tr_Y

def dataset_shuffle_train_data(self, size):
    self.indices = np.arange(size)
    np.random.shuffle(self.indices)

Dataset_torch.get_train_data = dataset_get_train_data
Dataset_torch.shuffle_train_data = dataset_shuffle_train_data

In [ ]:
def get_test_data(self):
    return self.te_xs,self.te_ys

Dataset_torch.get_test_data = get_test_data

In [ ]:
def get_validate_data(self, count):
    self.va_indices=np.arange(len(self.va_xs))
    np.random.shuffle(self.va_indices)
    
    va_X = self.va_xs[self.va_indices[0:count]]
    va_Y = self.va_ys[self.va_indices[0:count]]
    
    return va_X, va_Y

Dataset_torch.get_validate_date = get_validate_data
Dataset_torch.get_visualize_data = get_validate_data

In [20]:
def dataset_shuffle_data(self, xs, ys, tr_ratio=0.8, va_ratio=0.1,sqc=False):
    data_count = len(xs)
    tr_cnt = int(data_count * tr_ratio / 10) * 10
    va_cnt = int(data_count * va_ratio)
    te_cnt = data_count - (tr_cnt + va_cnt)

    tr_from, tr_to = 0, tr_cnt
    va_from, va_to = tr_cnt, tr_cnt + va_cnt
    te_from, te_to = tr_cnt + va_cnt, data_count

    indices = np.arange(data_count)
    np.random.shuffle(indices)
    self.tr_xs = xs[indices[tr_from:tr_to]]
    self.tr_ys = ys[indices[tr_from:tr_to]]
    self.va_xs = xs[indices[va_from:va_to]]
    self.va_ys = ys[indices[va_from:va_to]]
    self.te_xs = xs[indices[te_from:te_to]]
    self.te_ys = ys[indices[te_from:te_to]]
        
    self.input_shape = xs[0].shape
    self.output_shape = ys[0].shape
    
    return indices[tr_from:tr_to], indices[va_from:va_to], indices[te_from:te_to]

Dataset_torch.shuffle_data = dataset_shuffle_data

In [ ]:
def dataset_forward_postproc(self, loss_func, output, y_train, mode=None):
    if mode == None : mode = self.mode
    if mode == 'regression':
        loss = loss_func(output, y_train)
    elif mode == 'binary':
        loss = loss_func(output, y_train)
    elif mode == 'select':
        loss = loss_func(output, torch.argmax(y_train, axis=1))

#         entropy = softmax_cross_entropy_with_logits(y_train, output)
#         loss = torch.mean(entropy)

        
    return loss

Dataset_torch.forward_postproc = dataset_forward_postproc

In [25]:
def dataset_eval_accuracy_torch(self, x, y, output, mode =None):
    
    if mode is None: mode = self.mode
        
    if mode == 'regression':
        mse = torch.mean(torch.square(output - y))
        accuracy = 1 - torch.sqrt(mse) / torch.mean(y)
        
    elif mode == 'binary':
        estimate = torch.greater(output, -3)
        answer = torch.eq(y, torch.tensor(1.0))
        correct = torch.eq(estimate, torch.tensor(answer))
        accuracy = torch.mean(torch.tensor(correct).float())
        
    elif mode == 'select':
        estimate = torch.argmax(output, axis=1)
        answer = torch.argmax(y, axis=1)
        correct = torch.eq(estimate, answer)
        accuracy = torch.mean(correct.float())
        
    return accuracy

Dataset_torch.eval_accuracy = dataset_eval_accuracy_torch

In [ ]:
def dataset_get_estimate(self, output, mode=None):
    if mode is None: mode = self.mode
        
    if mode == 'regression':
        estimate = output
        
    elif mode == 'binary':
        estimate = F.sigmoid(output)
        
    elif mode == 'select':
        estimate = log_softmax_2_prob_torch(output)
#         estimate = softmax(output)
        
    return estimate

Dataset_torch.get_estimate = dataset_get_estimate

In [4]:
def dataset_train_prt_result(self, epoch, costs, accs, acc, time1, time2):
    costs=torch.tensor(costs)
    accs = torch.tensor(accs)
    print('    Epoch {}: cost={:5.3f}, accuracy={:5.3f}/{:5.3f} ({}/{} secs)'. \
          format(epoch, torch.mean(costs), torch.mean(accs), acc, time1, time2))

def dataset_test_prt_result(self, name, acc, time):
    print('Model {} test report: accuracy = {:5.3f}, ({} secs)\n'. \
          format(name, acc, time))

Dataset_torch.train_prt_result = dataset_train_prt_result
Dataset_torch.test_prt_result = dataset_test_prt_result

NameError: name 'Dataset_torch' is not defined

In [21]:
class Dataset_2_torch(Dataset):
    def __init__(self,x, y, len_):
        self.len_ = len_
        self.X_data  = torch.from_numpy(x).float()
        self.y_data = torch.from_numpy(y).float()

    def __len__(self):
        return self.len_

    def __getitem__(self,idx):
        
        x_ = self.X_data[idx]
        y_ = self.y_data[idx]

        return x_, y_

In [24]:
def dataloader_torch(self, batch_size, num_workers, len_):
    Ds=Dataset_2_torch(self.tr_xs, self.tr_ys, len_)
    dataloader = DataLoader(Ds, batch_size=batch_size, \
                            shuffle=True, num_workers=num_workers)
    return dataloader

Dataset_torch.get_train_loader = dataloader_torch